In [405]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [406]:
from CalcBipartisanScores import *

In [243]:
CONN_INFO

{'db': 'DDDB2016Aug',
 'host': 'dddb.chzg5zpujwmo.us-west-2.rds.amazonaws.com',
 'passwd': 'D1d2d3b4',
 'port': 3306,
 'user': 'awsDB'}

In [397]:
cnxn = pymysql.connect(**CONN_INFO)

votes_df = get_passing_votes(cnxn)

cnxn.close()

In [399]:
len(votes_df), len(votes_df[['pid', 'session_year']].drop_duplicates())

(499601, 241)

In [400]:
votes_df = get_full_vote_info(votes_df)

In [401]:
len(votes_df), len(votes_df[['pid', 'session_year']].drop_duplicates())

(499601, 241)

In [402]:
filters = ['unanimous', 'abstain_vote', 'resolution']
leg_scores_df = generate_full_scores_df(votes_df, filters)

Combo ()
2017-08-30 10:37:23.342535


C:\Users\andre\Documents\GitHub\dd-Data3.0\BipartisanScores\CalcBipartisanScores.py:162: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  votes_df['vote_score'] = votes_df.apply(calc_vote_score, axis=1)
C:\Users\andre\Documents\GitHub\dd-Data3.0\BipartisanScores\CalcBipartisanScores.py:129: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['normed_score'] = (s - s.min()) / (s.max() - s.min())


Combo ('unanimous',)
2017-08-30 10:39:50.615300
Combo ('abstain_vote',)
2017-08-30 10:42:17.494472
Combo ('resolution',)
2017-08-30 10:44:43.137859
Combo ('unanimous', 'abstain_vote')
2017-08-30 10:47:09.926946
Combo ('unanimous', 'resolution')
2017-08-30 10:49:33.628031
Combo ('abstain_vote', 'resolution')
2017-08-30 10:52:00.087636
Combo ('unanimous', 'abstain_vote', 'resolution')
2017-08-30 10:54:24.498955


In [407]:
cnxn = pymysql.connect(**CONN_INFO)

In [408]:
cols = ['pid', 'session_year', 'normed_score']
leg_scores_df[cols].rename(columns={'normed_score': 'score'}).to_sql('BipartisanshipScores',
                                                                     cnxn,
                                                                     'mysql',
                                                                     if_exists='replace',
                                                                     index=False)

C:\Program Files\Anaconda3\lib\site-packages\pandas\core\generic.py:1165: FutureWarning: The 'mysql' flavor with DBAPI connection is deprecated and will be removed in future versions. MySQL will be further supported with SQLAlchemy connectables.
  chunksize=chunksize, dtype=dtype)


In [409]:

cnxn.close()